In [ ]:
%matplotlib inline
import numpy as np                                                                                                 
import matplotlib.pyplot as plt                                                                                    
import pandas as pd

import bc_utils as butils

from scipy.spatial import distance
from mpl_toolkits import mplot3d

from sklearn.model_selection import train_test_split                                                               
from sklearn.preprocessing import StandardScaler                                                                   
from sklearn.neighbors import KNeighborsClassifier 

# 1. Load data

In [ ]:
# Read dataset to pandas dataframe
# current datasets ready for testing:
#   blobs.csv
#   blobs_3d.csv
#   iris.csv
#   mfeat-mor.csv
#   mfeat-zer.csv
#   mfeat-pix.csv
#   mfeat-fac.csv
#   mfeat-fou.csv
#   mfeat-kar.csv
#   noisy_circles.csv
#   noisy_circles_3d.csv
#   overlap.csv
#   pendigit.csv

df = pd.read_csv("data/pendigit.csv", index_col=0)

# 2. Plot data

Visualize data if they have 2 or 3 features.

In [ ]:
grouped = df.groupby('cluster')
for group in grouped:
    print(group)

In [ ]:
colors = {0:'red', 1:'blue', 2:'green'}

if (df.columns.size == 3):
    fix, ax = plt.subplots()
    grouped = df.groupby('cluster')
    
    i = 0
    for key, group in grouped:
        group.plot(ax=ax, kind='scatter', x='x', y='y',
                   label=key, color=colors[i])
        i = i + 1
    plt.show()
elif (df.columns.size == 4):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    grouped = df.groupby('cluster')

    i = 0
    for key, group in grouped:
        ax.scatter(group['x'], group['y'], group['z'], color=colors[i])
        i = i + 1
    plt.show()
else:
    print("There is no good visualisation for this dataset - " + str(df.columns.size) + " features.")
    

In [ ]:
#creating test datasets for iris
names = set(df['cluster'])

dataframes = {}
for name in names:
    tmp_df = df[df['cluster'] == name]
    dataframes[name] = butils.TestDf(tmp_df)
    
full_test_df = pd.DataFrame()
for name in names:
    full_test_df = full_test_df.append(dataframes[name].test_df)

In [ ]:
#using delta_medoids_full algorithm
results = {}
res_len = []
for name in names:
    delta_df = dataframes[name].train_df.iloc[:, :-1]

    result = butils.delta_medoids_full(delta_df, butils.estimate_delta(delta_df, distance.euclidean) , distance.euclidean)
    result['cluster'] = name
    results[name] = result
    res_len.append(result.shape[0])

print(res_len)

In [ ]:
#using delta_medoids_one_shot algorithm
results2 = {}
res_2_len = []
for name in names:
    delta_df = dataframes[name].train_df.iloc[:, :-1]
    result = butils.delta_medoids_one_shot(delta_df, butils.estimate_delta(delta_df, distance.euclidean), distance.euclidean)
    result['cluster'] = name
    results2[name] = result
    res_2_len.append(result.shape[0])
print(res_2_len)

In [ ]:
#using random select algorithm #TODO use same numbers as the best method found
results3 = {}
res_3_len = []
for name, len_num in zip(names, res_len):
    delta_df = dataframes[name].train_df.iloc[:, :-1]
    result = butils.random_select(delta_df, len_num)
    result['cluster'] = name
    results3[name] = result
    res_3_len.append(result.shape[0])
print(res_3_len)

In [ ]:
#using greedy algorithm
results4 = {}
res_4_len = []
for name in names:
    delta_df = dataframes[name].train_df.iloc[:, :-1]
    result = butils.greedy_select(delta_df, butils.estimate_delta(delta_df, distance.euclidean), distance.euclidean)
    result['cluster'] = name
    results4[name] = result
    res_4_len.append(result.shape[0])
print(res_4_len)

In [ ]:
#creating training DataFrames for comparing oneshot and full delta medoids algorithm
train_delta_medoids_full = pd.DataFrame()
train_delta_medoids_one_shot = pd.DataFrame()
train_random_selection = pd.DataFrame()
train_greedy_select = pd.DataFrame()

for name in names:
    train_delta_medoids_full = train_delta_medoids_full.append(results[name])
    train_delta_medoids_one_shot = train_delta_medoids_one_shot.append(results2[name])
    train_random_selection = train_random_selection.append(results3[name])
    train_greedy_select = train_greedy_select.append(results4[name])

In [ ]:
def classifyPoints(ref_df, test_df):

    X_train = ref_df.iloc[:, :-1].values
    y_train = ref_df.iloc[:, -1].values
    X_test = test_df.iloc[:, :-1].values
    y_test = test_df.iloc[:, -1].values

    from sklearn.preprocessing import StandardScaler  
    scaler = StandardScaler()  
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)  
    X_test = scaler.transform(X_test)

    from sklearn.neighbors import KNeighborsClassifier  
    classifier = KNeighborsClassifier(n_neighbors=1)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    from sklearn.metrics import classification_report, confusion_matrix
    conf_mat = confusion_matrix(y_test, y_pred)
    print(conf_mat)  
    print(classification_report(y_test, y_pred))
    return conf_mat

In [ ]:
matrix_full = classifyPoints(train_delta_medoids_full, full_test_df)

In [ ]:
matrix_one_shot = classifyPoints(train_delta_medoids_one_shot, full_test_df)

In [ ]:
matrix_random_selection = classifyPoints(train_random_selection, full_test_df)

In [ ]:
matrix_greedy_select = classifyPoints(train_greedy_select, full_test_df)

In [ ]:
def benchmark_result_plots(matrices, titles, headers):
    """
    parameters:
    matrices - list of 2d np.array
    titles - list of graph titles
    names - list of labels
    """
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(15,15))
    ax_list = [ax1, ax2, ax3, ax4]

    for index in range(0, len(matrices)):
        tick_labels = sorted(list(headers[index]))
        ax = ax_list[index]
        ax.imshow(matrices[index], cmap='binary')

        # We want to show all ticks...
        ax.set_xticks(np.arange(len(tick_labels)))
        ax.set_yticks(np.arange(len(tick_labels)))

        # ... and label them with the respective list entries
        ax.set_xticklabels(tick_labels)
        ax.set_yticklabels(tick_labels)

        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), ha="right")

        # Loop over data dimensions and create text annotations.
        for i in range(len(tick_labels)):
            for j in range(len(tick_labels)):
                if(matrices[index][i, j] == 0):
                    continue
                if i == j:
                    text = ax.text(j, i, matrices[index][i, j], ha="center", va="center", color="g",
                                   weight="bold", fontsize=16)
                    continue
                else:
                    text = ax.text(j, i, matrices[index][i, j], ha="center", va="center", color = "r",
                                   weight="bold", fontsize=16)
            
        ax.set_title(titles[index], fontsize=16)
        
benchmark_result_plots([matrix_full, matrix_one_shot, matrix_random_selection, matrix_greedy_select],
                       ['Delta-Medoids Full', 'Delta-Medoids One Shot', 'Random Selection', 'Greedy Selection'],
                       [names, names, names, names])